# Data Wrangling, Data Intuition & Wisdom of the Crowds

Putting it all together:
- pandas
- data wrangling
- data intuition
- wisdom of the crowds
- EDA

We'll use your responses to the Fermi estimation Google Form to do this.

## Setup

In [1]:
# import working with data libraries
import pandas as pd
import numpy as np

## The Data

In [2]:
# read data into Python
df = pd.read_csv('data/fermi_wi21.csv')

In [3]:
# take a look at the data
df

,Timestamp,How fast does human hair grow (cm/yr)?,"If every living person stood crammed together side-by-side, how large of an area would they occupy (km²)?","How many days would it take to walk from here to New York City (assuming no stopping to fix shoes, apply sunscreen, or for sleeping, eating, or other biological needs)?"
0,1/28/21 13:35,10 cm/1 year,"10,000",70
1,1/29/21 9:51,10,100000,100
2,1/29/21 9:51,10,100000,100
3,1/29/21 9:51,3,400,10 days
4,1/29/21 9:51,5,"1,000,000,000",30
...,...,...,...,...
78,1/31/21 14:43,25,1.2,100
79,1/31/21 18:54,100,1000,100
80,1/31/21 19:13,10,1000000,100000
81,1/31/21 20:41,5,25,4


## Data Cleaning & Wrangling

Tidy Data Rules (Review):
1. Every observation in a row
2. Every variable in a column
3. If multiple tables, column on which to merge

### iClicker Question #1

Are these data in the tidy data format?

- A) Yes, these data are ready to analyze
- B) Yes, but there is more work to do before analysis
- C) No, not tidy
- D) Have no idea what you're talking about

## Brainstorming

What considerations do we have to make about these data?

- handling non-numeric answers (i.e. "lots")
- handle units in answer
- unreasonably large/small number (outliers)? (not thoughtful, or joking)
- scientific notation

In [4]:
# change column names
df.columns = ['timestamp', 'hair_growth', 'crammed', 'SAN_NYC']
df.head()

,timestamp,hair_growth,crammed,SAN_NYC
0,1/28/21 13:35,10 cm/1 year,"10,000",70
1,1/29/21 9:51,10,100000,100
2,1/29/21 9:51,10,100000,100
3,1/29/21 9:51,3,400,10 days
4,1/29/21 9:51,5,"1,000,000,000",30


In [5]:
# check type of each Series (column)
df.dtypes

timestamp      object
hair_growth    object
crammed        object
SAN_NYC        object
dtype: object

In [6]:
# this cell will produce an error
df['hair_growth'].median()

TypeError: 

## Cleaning: Hair Growth

How fast does human hair grow (cm/yr)?

### iClicker Question #2

Which do you think is closest the actual speed at which hair grows (in cm/yr)?

- A) 1
- B) 10
- C) 100
- D) 1000

In [7]:
# take a look at unique values
df["hair_growth"].unique()

array(['10 cm/1 year', '10', '3', '5', '100', '2', '20', '0.5/1',
       '8cm/1yr', '15', '60', '30', '12cm per year', '1.25', '20 cm/yr',
       '12cm/yr', '21', '100 cm/year ', '10cm/yr', '24', '36',
       '12.7 centimeters', '6 inches a year', '12', '110',
       "6 My hair doesn't grow that much in a year", '8', '2-3cm/yr',
       '15.24', '4', '2 cm/yr', '25/1.5', '10^2cm/yr', '7', '1000',
       '60 cm/yr', '1.1', '10^2', '25', '23'], dtype=object)

In [8]:
# standardize height column
def standardize_hair(string):
    
    # Basic string pre-processing
    string = string.lower()
    string = string.strip()
    
    # take care of some extra info
    string = string.replace("my hair doesn't grow that much in a year", "")
    string = string.replace("25/1.5", "16.67")

    # wrong unit used
    # 1 in == 2.54 cm
    string = string.replace("6 inches a year", "15.24")
    
    # range provided
    string = string.replace("2-3", "2.5")
    

    # take care of inclded unit cases        
    string = string.replace("cm/year", "")
    string = string.replace("cm/ye", "")
    string = string.replace("centimeters", "")
    string = string.replace("cm per year", "")
    string = string.replace("cm/yr", "")
    string = string.replace("cm/1yr", "")
    string = string.replace("cm/1 year", "")
    string = string.replace("yr", "")
    string = string.replace("cm", "")    
    string = string.replace("/", "")

    string = string.replace("^2", "0")
    
    string = string.strip()

    # convert to numeric
    output = float(string)
    
    return output

In [9]:
# apply function across values in hair growth columns
df["hair_growth"] = df["hair_growth"].apply(standardize_hair)
df["hair_growth"].unique()

array([1.000e+01, 3.000e+00, 5.000e+00, 1.000e+02, 2.000e+00, 2.000e+01,
       5.100e-01, 8.000e+00, 1.500e+01, 6.000e+01, 3.000e+01, 1.200e+01,
       1.250e+00, 2.100e+01, 2.400e+01, 3.600e+01, 1.270e+01, 1.524e+01,
       1.100e+02, 6.000e+00, 2.500e+00, 4.000e+00, 1.667e+01, 7.000e+00,
       1.000e+03, 1.100e+00, 2.500e+01, 2.300e+01])

In [10]:
df['hair_growth'].describe()

count      83.000000
mean       38.641084
std       111.355994
min         0.510000
25%        10.000000
50%        12.700000
75%        30.000000
max      1000.000000
Name: hair_growth, dtype: float64

In [11]:
# compare to actual value: 15 cm/year (~6 in)
df["hair_growth"].median()

12.7

## Cleaning: Crammed

If every living person stood crammed together side-by-side, how large of an area would they occupy (km²)?

### iClicker Question #3

Which do you think is closest the actual area every living person would occupy (in km²)?

- A) 100
- B) 1,000
- C) 10,000
- D) 100,000

In [ ]:
df['crammed'].unique()

In [ ]:
# standardize crammed column
# code from last quarter - needs to be updated
def standardize_crammed(string):
    
    orig = string
    output = None
    
    # Basic string pre-processing
    string = string.lower()
    string = string.strip()
    
    # take care of commas
    string = string.replace(",", "")

    # take care of inclded unit cases
    string = string.replace("(km^2)", "")
    string = string.replace("km^2", "")
    string = string.replace("km²", "")
    string = string.replace("km", "")

    string = string.strip()
    
    # take care of scientific notation / word cases
    string = string.replace("x10^10", "0000000000")
    string = string.replace("7.5 x 10^6", "7500000")
    string = string.replace("x10^2", "00")
    string = string.replace(" million", "000000")
    
    # take care of case with space in middle
    string = string.replace("100 000", "100000")


    # convert to numeric
    output = float(string)
    
    return output

In [ ]:
# apply function across values in crammed columns
df["crammed"] = df["crammed"].apply(standardize_crammed)
df["crammed"].unique()

In [ ]:
# check to see how commas are being handled
df.dtypes

In [ ]:
# compare to actual value: 1,000-10,000 km^2)
df['crammed'].median()

## Cleaning: San Diego to NYC

How many days would it take to walk from here to New York City (assuming no stopping to fix shoes, apply sunscreen, or for sleeping, eating, or other biological needs)?

In [ ]:
df['SAN_NYC'].unique()

In [ ]:
# standardize distance column
# code from last quarter - needs to be updated
def standardize_distance(string):
    
    orig = string
    output = None
    
    # Basic string pre-processing
    string = string.lower()
    string = string.strip()
    
    # remove units
    string = string.replace("days", "")
    
    # take care of non-days answers
    string = string.replace("4 months", "120")
    string = string.replace("1 year", "365")
    string = string.replace("lots", "NaN")
    
    # take care of scientific notation 
    string = string.replace("10^3", "10000")
    
    string = string.strip()

    # convert to numeric
    output = float(string)
    
    return output

In [ ]:
# apply function across values in crammed columns
df["SAN_NYC"] = df["SAN_NYC"].apply(standardize_distance)
df["SAN_NYC"].unique()

In [ ]:
# compare to actual value: 38 days)
df['SAN_NYC'].median()

## Take Home Messages

1. Look at your data! (People behave unexpectedly.)
2. Get your data into a tidy data format.
3. Fix (Standardize) your data - or, better yet, collect better data!
4. Use your intuition/knowledge/information out there to consider if values make sense.

**Improvements** (your classmates' suggestsions)

### Crammed

In [12]:
def cram_helper(cram):
    if len(cram) == 1:
        return cram[0]
    if cram[1] == '*':
        return str(float(cram[0]) * float(cram[2]))

    
df["crammed_ss"] = pd.to_numeric(df["crammed"].
                              str.lower().
                              replace({",":"",
                                       "\([\(\w /\.~\^)=?\']+":"1633.5", 
                                       "km[\^]?2":"", "\^":"e", 
                                       " million": " * 1000000", 
                                       "^[a-z]+$":"", "[x]": " * "}, 
                                      regex = True).
                              str.strip().
                              str.split(" ").
                              apply(cram_helper), 
                              errors = 'coerce')

In [13]:
# standardize crammed column
def standardize_crammed_df(string):

    orig = string
    output = None

# Basic string pre-processing
    string = string.lower()
    string = string.strip()

# take care of commas
    string = string.replace(",", "")

# take care of inclded unit cases
    string = string.replace("km2", "")
    string = string.replace("km^2", "")

    string = string.strip()

# take care of scientific notation / word cases
    string = string.replace("x10^8", "00000000")
    string = string.replace("*10^12", "000000000000")
    string = string.replace("10^6", "1000000")
    string = string.replace("1.3x10^3", "1300")
    string = string.replace("10^5", "100000")
    string = string.replace("3.5 million", "3500000")

# take care of invalid entries (why would you do this)
    string = string.replace("idk", "NaN")
    string = string.replace("texas", "NaN")

# take care of that one really long string that no one asked for
    string = string.replace("(sqr(7.5 billion people) * 0.00033 m) * (sqr(7.5 billion people) * 0.00066 m) ~ 1633.5  (a rectangle of people 'cause our sides are shorter than our fronts/backs???)", "1633.5")

# convert to numeric
    output = float(string)

    return output

In [14]:
def standardize_crammed_ai(string):

    output = None

    # Basic string pre-processing + take care of commas
    string = string.lower().strip().replace(",", "")

    # take care of included unit cases
    string = string.replace("km^2", "").strip()
    string = string.replace("km2", "").strip()

    # take care of special cases
    string = string.replace("idk", "")
    string = string.replace("(sqr(7.5 billion people) * 0.00033 m) * (sqr(7.5 billion people) * 0.00066 m) ~ 1633.5  (a rectangle of people 'cause our sides are shorter than our fronts/backs???)", "1633.5")
    string = string.replace("texas", "695663")

    # take care of scientific notation and word cases with math/eval
    string = string.replace(" million", "1000000")
    string = string.replace("x", "").replace("^", "**")

    # convert to numeric
    try:
        output = float(eval(string))
    except:
        output = np.nan

    return output

In [15]:
# standardize crammed column
def standardize_crammed_tc(string):

    string = string.lower()
    num_list = []

    #Remove answers in words
    if not string[0] in [str(x) for x in range(10)]:
        return None

    # Remove unnecessary char
    if "," in string:
        string = string.replace(",", "")
    if " " in string:
        string = string.replace(" ", "")
    if "km" in string:
        string = string.split("km")[0]

    # Solve outliner "million"
    if "million" in string:
        string = string.replace("million", "x10^6")

    # Separate numbers to be multiplied
    if "x" in string:
        num_list = string.split("x", 1)
    elif "" in string:
        num_list = string.split("", 1)
    else:
        num_list.append(string)

    # Calculate scientific notation
    if "^" in num_list[-1]:
        ten, power = num_list[-1].split("^", 1)
        num_list[-1] = float(ten) ** float(power)

    # Construct number
    if len(num_list) == 1:
        output = float(num_list[0])
    else:
        output = float(num_list[0]) * float(num_list[1])
    
    return output

In [16]:
df["crammed_ai"] = df["crammed"].apply(standardize_crammed_ai)
df["crammed_df"] = df["crammed"].apply(standardize_crammed_df)

In [17]:
# compare to actual value: 1,000-10,000 km^2)
print(df['crammed_ss'].median(), df['crammed_df'].median(), df['crammed_ai'].median())

100000.0 100000.0 99999.5


In [18]:
# starting to look at places where values differ
# see cases not handled by one or the other method
df[df['crammed_ss'] != df['crammed_ai']]

,timestamp,hair_growth,crammed,SAN_NYC,crammed_ss,crammed_ai,crammed_df
17,1/29/21 9:52,60.00,3.5 million,1 year,3.500000e+06,3.510000e+00,3.500000e+06
29,1/29/21 9:53,20.00,7x10^8,40,7.000000e+09,6.457535e+22,7.000000e+08
31,1/29/21 9:54,24.00,4*10^12,2000,NaN,4.000000e+12,4.000000e+12
36,1/29/21 9:54,15.24,idk,20,NaN,NaN,NaN
48,1/29/21 9:58,6.00,(sqr(7.5 billion people) * 0.00033 m) * (sqr(7...,(3000 mi * 20 min/per mile)/60 min = 1000 hrs???,NaN,1.633500e+03,1.633500e+03
63,1/29/21 18:46,100.00,10^6 km^2,10^3 days,1.000000e+07,1.000000e+06,1.000000e+06
65,1/29/21 21:27,10.00,Texas,73,NaN,6.956630e+05,NaN
68,1/29/21 23:16,60.00,1.3x10^3,60,1.300000e+04,2.248091e+00,1.300000e+03
73,1/30/21 9:02,100.00,10^5,10^2,1.000000e+06,1.000000e+05,1.000000e+05


In [19]:
df[df['crammed_ss'] != df['crammed_df']]

,timestamp,hair_growth,crammed,SAN_NYC,crammed_ss,crammed_ai,crammed_df
29,1/29/21 9:53,20.00,7x10^8,40,7.000000e+09,6.457535e+22,7.000000e+08
31,1/29/21 9:54,24.00,4*10^12,2000,NaN,4.000000e+12,4.000000e+12
36,1/29/21 9:54,15.24,idk,20,NaN,NaN,NaN
48,1/29/21 9:58,6.00,(sqr(7.5 billion people) * 0.00033 m) * (sqr(7...,(3000 mi * 20 min/per mile)/60 min = 1000 hrs???,NaN,1.633500e+03,1.633500e+03
63,1/29/21 18:46,100.00,10^6 km^2,10^3 days,1.000000e+07,1.000000e+06,1.000000e+06
65,1/29/21 21:27,10.00,Texas,73,NaN,6.956630e+05,NaN
68,1/29/21 23:16,60.00,1.3x10^3,60,1.300000e+04,2.248091e+00,1.300000e+03
73,1/30/21 9:02,100.00,10^5,10^2,1.000000e+06,1.000000e+05,1.000000e+05


In [32]:
df[df['crammed_ai'] != df['crammed_df']]

,timestamp,hair_growth,crammed,SAN_NYC,crammed_ss,crammed_ai,crammed_df
17,1/29/21 9:52,60.00,3.5 million,1 year,3.500000e+06,3.510000e+00,3500000.0
29,1/29/21 9:53,20.00,7x10^8,40,7.000000e+09,6.457535e+22,700000000.0
36,1/29/21 9:54,15.24,idk,20,NaN,NaN,NaN
65,1/29/21 21:27,10.00,Texas,73,NaN,6.956630e+05,NaN
68,1/29/21 23:16,60.00,1.3x10^3,60,1.300000e+04,2.248091e+00,1300.0


### San Diego to NYC

In [20]:
def dist_helper(distance):
    if len(distance) == 1:
        return distance[0]
    if distance[1] == '*':
        return str(float(distance[0]) * float(distance[2]))
    if distance[1] == '/':
        return str(float(distance[0]) / float(distance[2]))

df["SAN_NYC_ss"] = pd.to_numeric(df["SAN_NYC"].
                              str.lower().
                              replace({"days?":"", 
                                       "\^":"e", "^\([\w */)=?]+$":"1000", 
                                       " months?":" * 30", 
                                       " hours?": " / 24", 
                                       " years?": " * 365"}, regex = True).
                              str.strip().
                              str.split(' ').
                              apply(dist_helper), errors = 'coerce')

In [21]:
# standardize distance column
def standardize_distance_df(string):

    orig = string
    output = None

    # Basic string pre-processing
    string = string.lower()
    string = string.strip()

    # remove units
    string = string.replace("days", "")

    # take care of non-days answers
    string = string.replace("1 month", "30")
    string = string.replace("1 year", "365")
    string = string.replace("2 years", "730")
    string = string.replace("1000 hours", "41.7")
    string = string.replace("8 months", "240")

    # take care of scientific notation 
    string = string.replace("10^3", "10000")
    string = string.replace("10^2", "1000")

    # take care of that one really long string no one asked for
    string = string.replace("(3000 mi * 20 min/per mile)/60 min = 1000 hrs???", "41.7")

    string = string.strip()

    # convert to numeric
    output = float(string)

    return output

In [22]:
def standardize_distance_ai(string):

    output = None

    # Basic string pre-processing
    string = string.lower().strip()

    # remove units
    string = string.replace("days", "").strip()

    # special case
    string = string.replace("(3000 mi * 20 min/per mile)/60 min = 1000 hrs???", "1000 hours")

    # take care of non-days answers (strip 's' for plurals)
    string = string.replace("month", "*30").strip(' s')
    string = string.replace("year", "*365").strip(' s')
    string = string.replace("hour", "/24").strip(' s')

    # take care of scientific notation 
    string = string.replace("^", "**")

    # convert to numeric
    try:
        output = float(eval(string))
    except:
        output = np.nan

    return output

In [23]:
df["SAN_NYC_ai"] = df["SAN_NYC"].apply(standardize_distance_ai)

In [24]:
# compare to actual value: 38 days
print(df['SAN_NYC_ss'].median(), df['SAN_NYC_ai'].median())

87.0 85.0


In [25]:
# where values differ between two approaches
df[df['SAN_NYC_ss'] != df['SAN_NYC_ai']]

,timestamp,hair_growth,crammed,SAN_NYC,crammed_ss,crammed_ai,crammed_df,SAN_NYC_ss,SAN_NYC_ai
48,1/29/21 9:58,6.0,(sqr(7.5 billion people) * 0.00033 m) * (sqr(7...,(3000 mi * 20 min/per mile)/60 min = 1000 hrs???,NaN,1633.5,1633.5,1000.000000,41.666667
50,1/29/21 10:01,2.5,"1,000,000",1000 hours,1000000.0,1000000.0,1000000.0,41.666667,41.666667
63,1/29/21 18:46,100.0,10^6 km^2,10^3 days,10000000.0,1000000.0,1000000.0,10000.000000,1000.000000
73,1/30/21 9:02,100.0,10^5,10^2,1000000.0,100000.0,100000.0,1000.000000,100.000000


The test: how do new functions work on last year's data (different specific responses)

## Crammed

In [30]:
df_fa20 = pd.read_csv('data/fermi_fa20.csv')
df_fa20.columns = ['timestamp', 'hair_growth', 'crammed', 'SAN_NYC']
df_fa20

,timestamp,hair_growth,crammed,SAN_NYC
0,10/10/20 2:37,20,3500000,10
1,10/10/20 13:16,25,1 million,50
2,10/10/20 13:45,10^3 cm/yr,10^7 km^2,10^4
3,10/10/20 13:52,6cm/yr,1000km^2,200 days
4,10/10/20 14:18,81,16875000000,28000
...,...,...,...,...
186,10/16/20 18:00,12,650,"42 (I'm assuming ""here"" means ucsd)"
187,10/16/20 20:05,10,2500000,1000
188,10/16/20 22:32,7,100000,240
189,10/16/20 23:04,20,500000,40


In [32]:
df_fa20["crammed_ai"] = df_fa20["crammed"].apply(standardize_crammed_ai)
# fails df_fa20["crammed_df"] = df_fa20["crammed"].apply(standardize_crammed_df)

df_fa20["crammed_ss"] = pd.to_numeric(df_fa20["crammed"].
                              str.lower().
                              replace({",":"",
                                       "\([\(\w /\.~\^)=?\']+":"1633.5", 
                                       "km[\^]?2":"", "\^":"e", 
                                       " million": " * 1000000", 
                                       "^[a-z]+$":"", "[x]": " * "}, 
                                      regex = True).
                              str.strip().
                              str.split(" ").
                              apply(cram_helper), 
                              errors = 'coerce')

In [33]:
print(df_fa20['crammed_ss'].median(), df_fa20['crammed_ai'].median())

100000.0 100000.0


### SAN TO NYC

In [34]:
df_fa20["SAN_NYC_ss"] = pd.to_numeric(df_fa20["SAN_NYC"].
                              str.lower().
                              replace({"days?":"", 
                                       "\^":"e", "^\([\w */)=?]+$":"1000", 
                                       " months?":" * 30", 
                                       " hours?": " / 24", 
                                       " years?": " * 365"}, regex = True).
                              str.strip().
                              str.split(' ').
                              apply(dist_helper), errors = 'coerce')
df_fa20["SAN_NYC_ai"] = df_fa20["SAN_NYC"].apply(standardize_distance_ai)

In [35]:
print(df_fa20['SAN_NYC_ss'].median(), df_fa20['SAN_NYC_ai'].median())

69.5 69.5
